prep_test_data.py -- contact: Jay Sayre, jsayre@ucdavis.edu

Python code to create a sample two-period consumption dataset based on the World Bank Global Consumption Database 2010 found here (I cannot distribute this myself, but it is publicly available): 
https://microdata.worldbank.org/index.php/catalog/4424/data-dictionary/f01?file_name=WB_GCD_2010_v2014-03_survey_data.xlsx (WLD_GCD_2010_v2014-03_M)

In [1]:
import os
import pandas as pd
import numpy as np
np.random.seed(12345)

### Directories    

### Inputs ###
lsms_data                      =  os.path.join("./",  "WB_GCD_2010_v2014-03_data.xlsx")
created_cons_dta               =  os.path.join("./",  "lsms_test_data.dta")


In [2]:
#### Write excel to stata data file
df = pd.read_excel(lsms_data)
df = df[df['Indicatorname'].isin(['Annual expenditure, per capita','Nb. of households in sample','Share of product/service in total expenditure (%)'])]

df.drop(['Codeofproductorservice','Sex','Agegroup','PPPconversionfactor','Surveyyear','Exchangerate'],axis=1,inplace=True)
df['Labelofproductorservice'] = df['Labelofproductorservice'].str.strip()

wrld_regs = df['Region'].unique()

inc_df = df[df['Indicatorname'] == 'Annual expenditure, per capita']
inc_df = inc_df[inc_df['Measurementunit'] == 'US$ (mean 2010 exchange rate)']
inc_df = inc_df[inc_df['Incomegroup'] != 'All']
inc_df = inc_df[['Countryname','SurveyID','Area','Incomegroup','Value']].reset_index(drop=True)
inc_df.columns = ['Countryname','SurveyID','area','income','outlays']

wght_df = df[df['Indicatorname'] == 'Nb. of households in sample']
wght_df = wght_df[wght_df['Incomegroup'] != 'All']
wght_df = wght_df[['Countryname','SurveyID','Area','Incomegroup','Value']].reset_index(drop=True)
wght_df.columns = ['Countryname','SurveyID','area','income','wt']

meat_prods = ['Beef and veal','Pork','Lamb, mutton and goat','Poultry',
              'Fresh, chilled or frozen fish and seafood','Preserved or processed fish and seafood']
other_animal_prods = ['Fresh milk','Preserved milk and other milk products','Eggs and egg-based products']

df = df[df['Indicatorname'] == 'Share of product/service in total expenditure (%)']
df = df[df['Labelofproductorservice'].isin(meat_prods+other_animal_prods)]
df = df[df['Incomegroup'] != 'All']
df = df[['Countrycode','Countryname','Region','SurveyID','Year','Labelofproductorservice','Area','Incomegroup','Value']]
df.columns = ['iso','Countryname','Region','SurveyID','year','prod','area','income','share']

df = df.merge(wght_df, on=['Countryname','SurveyID','area','income'], how='left')
df = df.merge(inc_df, on=['Countryname','SurveyID','area','income'], how='left')

prod_conv_df = {'Beef and veal':'Cow meat', 'Pork':'Pig meat', 'Lamb, mutton and goat':'Sheep and goat meat', 
 'Other meats and meat preparations':'Other meat', 'Eggs and egg-based products':'Poultry eggs',
 'Fresh, chilled or frozen fish and seafood':'Fish', 'Preserved or processed fish and seafood':'Fish',
 'Fresh milk':'Cow milk','Preserved milk and other milk products':'Cow milk'}

df['G_group'] = df['prod'].apply(lambda x: 1 if x in meat_prods else 2)
df['prod'] = df['prod'].apply(lambda x: prod_conv_df[x] if x in prod_conv_df.keys() else x)
df['market_id'] = df['Region'].factorize()[0]
region_mapping = dict(zip(range(len(df['Region'].factorize()[1])),df['Region'].factorize()[1]))
df['expenditure'] = df['share'] * df['outlays']
df.rename(columns={'year':'period_id','prod':'i_good','outlays':'exp_cap'}, inplace=True)
df['hh_id'] = df['Countryname']+'_'+df['area']+'_'+df['income']+'_'+df['SurveyID']
df.drop(['iso','Region','SurveyID','area','income','share','Countryname'],axis=1,inplace=True)
df = df[~df['exp_cap'].isna()]
df = df[~df['expenditure'].isna()]
df['hh_id'] = df['hh_id'].factorize()[0]
df['period_id'] = 1

### Run this for both to generate Engel curves across all goods, not just within group
total_exp = df.groupby(['period_id','hh_id','market_id'])['expenditure'].sum().reset_index()

total_exp_df = df[['period_id','hh_id','market_id','exp_cap','wt']].drop_duplicates().merge(total_exp, on=['period_id','hh_id','market_id'], how='left')
total_exp_df['exp_cap'] = total_exp_df.apply(lambda row: row['expenditure'] if row['expenditure'] > row['exp_cap'] else row['exp_cap'], axis=1)
total_exp_df['rem_exp'] = total_exp_df['exp_cap']-total_exp_df['expenditure']

df = df.drop('exp_cap',axis=1).merge(total_exp_df.drop(['rem_exp','wt','expenditure'],axis=1), on=['period_id','hh_id','market_id'], how='left')

total_exp_df.drop('expenditure',axis=1,inplace=True)
total_exp_df.rename(columns={'rem_exp':'expenditure'},inplace=True)
total_exp_df['i_good'] = 'Remainder'
total_exp_df['G_group'] = 3
df = pd.concat([df,total_exp_df])

### Generate a "period 2" dataset
df2 = df.copy()
df2['period_id'] = 2
df2['hh_id'] = df2['hh_id'] + df['hh_id'].max() + 1

exp_mult = {
    'Cow meat':            0.80,
    'Pig meat':            0.90,
    'Sheep and goat meat': 0.85,
    'Poultry':             0.75,
    'Fish':                0.95,
    'Cow milk':            1.00,
    'Poultry eggs':        0.90,
    'Remainder':           1.40
}

wt_mult = {h: np.random.uniform(0.01, 4) for h in df2['hh_id'].unique()}
df2['wt']=np.round(df2['wt']*df2['hh_id'].map(wt_mult), 0)

def bump_exp(row):
    m = exp_mult[row['i_good']]
    noise = np.random.normal(scale=0.05 * row['expenditure'])
    return row['expenditure'] * m + noise
df2['expenditure'] = df2.apply(bump_exp, axis=1)
df2['exp_cap'] = df2.groupby(['period_id','hh_id','market_id'])['expenditure'].transform('sum')
two_period_df = pd.concat([df,df2])

two_period_df = two_period_df[two_period_df['i_good']!='Remainder']
two_period_df.to_stata(created_cons_dta, write_index=False)